# Prerequisites & Setup

- Install Wireshark, tcpdump/tshark, and have Python (3.8+) with `scapy`, `pyshark`, and `requests` available in a virtualenv.
- Always run captures in an isolated lab environment (VMs or containers) and never on production networks without authorization.

**Commands:**
- tcpdump: `sudo tcpdump -i <iface> -w capture.pcap`
- tshark: `tshark -r capture.pcap -Y "http" -V`
- Wireshark: open `capture.pcap` and use display filters like `ip.addr == 10.0.0.5`

## Lab 1 — Packet capture basics

1. Start a capture:
```
sudo tcpdump -i <iface> -w lab1.pcap
```
2. Generate traffic (e.g., `curl https://example.com`).
3. Stop capture (Ctrl+C) and open `lab1.pcap` in Wireshark.

Useful filters:
- `http` — HTTP traffic
- `dns` — DNS queries
- `arp` — ARP traffic
- `tcp.analysis.retransmission` — TCP retransmissions

**Exercise:** Capture an HTTP request and follow TCP stream in Wireshark to observe the full request/response.

## Lab 2 — ARP observation & Scapy (safe)

- Observe ARP requests and responses with `sudo tcpdump -n -i <iface> arp` or Wireshark filter `arp`.

Python (Scapy) example to print ARP requests observed:
```python
from scapy.all import sniff, ARP

def handle(pkt):
    if pkt.haslayer(ARP) and pkt[ARP].op == 1:
        print(f"ARP who-has {pkt[ARP].pdst} from {pkt[ARP].psrc}")

sniff(filter='arp', prn=handle, count=50)
```

**Safety:** Do not send forged ARP replies on shared or production networks; perform any injection only within an isolated lab network with consent.

## Lab 3 — TCP handshake analysis

- Capture a TCP handshake (filter: `tcp.flags.syn == 1 || tcp.flags.ack == 1`). In Wireshark, `Analyze` → `Follow` → `TCP Stream` to view payload.

- Observe `seq` and `ack` numbers on the SYN, SYN-ACK, ACK packets. Induce packet loss in a controlled lab to observe retransmission behavior and `tcp.analysis.retransmission` events.

Python snippet to open a TCP connection and print socket options (example):
```python
import socket
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
try:
    s.settimeout(2.0)
    s.connect(('example.com', 80))
    s.send(b'GET / HTTP/1.0\r\nHost: example.com\r\n\r\n')
    print(s.recv(1024))
finally:
    s.close()
```


## Lab 4 — DNS analysis

- Use `dig +short example.com` and `dig @8.8.8.8 example.com` to test resolution and different resolvers.
- Capture DNS queries (filter: `dns`) and observe query ID, flags (QR, AA, TC, RD, RA), and records returned.

Exercise:
- Set up a local caching resolver (e.g., `dnsmasq`) and observe cache TTL behavior and how responses change when authoritative records change.


## Lab 5 — TLS inspection & checks

- Use `openssl s_client -connect example.com:443 -servername example.com` to view certificate chain and negotiated cipher.
- Use `testssl.sh` or `sslyze` for automated scanning of ciphers and protocol support.

Exercise:
- Scan a local webserver (lab VM) after configuring TLS parameters and observe how the server's supported ciphers change.


## Final exercises and next steps

- Combine packet capture with application logs (webserver, DNS server) to perform correlation exercises.
- Build a small flow-based anomaly detector using exported NetFlow data and a simple ML model (e.g., IsolationForest). See `../Python-Testing/python_examples.md` for pointers on ML ideas.

**Safety note:** Always perform active tests in a sandboxed lab network and get permission before interacting with third-party systems.

---

*If you'd like, I can expand any lab into a more detailed notebook or add pre-built sample PCAP files and automated checks.*